In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv(r'letras.txt', sep='\t', skiprows=0, header=None)
df.head(12)

,0,1,2,3,4
0,1.0,1.0,1.0,1.0,1.0
1,1.0,-1.0,-1.0,-1.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,-1.0,-1.0,-1.0,1.0
4,1.0,-1.0,-1.0,-1.0,1.0
5,NaN,NaN,NaN,NaN,NaN
6,1.0,1.0,1.0,1.0,1.0
7,1.0,-1.0,-1.0,-1.0,1.0
8,1.0,1.0,1.0,1.0,1.0
9,1.0,-1.0,-1.0,-1.0,1.0


Paso las matrices a vectores 

In [3]:
data= []
toAdd= []
for index, row in df.iterrows():
    for column in df:
        number= row[column]
        if (not pd.isna(number)):
            toAdd.append(int(number))
        else:
            data.append(toAdd)
            toAdd=[]
            break

print("L: ", data[11])
print("P: ", data[15])
print("T: ", data[19])
print("X: ", data[23])

L:  [1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]
P:  [1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1]
T:  [1, 1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1]
X:  [1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1]


Analizo la ortogonalidad de los vectores (más cercano a 0 significa más ortogonales)

In [24]:
print("L, X: ", np.dot(data[11], data[23]))
print("L, T: ", np.dot(data[11], data[19]))
print("L, P: ", np.dot(data[11], data[15]))

print("X, T: ", np.dot(data[23], data[19]))
print("X, P: ", np.dot(data[23], data[15]))

print("T, P: ", np.dot(data[19], data[15]))

L, X:  1
L, T:  -3
L, P:  -1
X, T:  1
X, P:  -5
T, P:  3


In [25]:
print("O Q: ", np.dot(data[14], data[16]))

O Q:  23


In [33]:
class Hopfield:
   
    # init method or constructor 
    def __init__(self, data):
        self.W= self.calculateSynapticWeight(data)
        
    def calculateSynapticWeight(self, data):
        firstRow= data.pop(0)
        Kt= np.array([firstRow])
        while len(data) > 0:
            newRow= np.array(data.pop(0))
            Kt= np.concatenate([Kt, [newRow]])
        K= Kt.T
        
        N= len(firstRow)
        w= (1/N) * np.dot(K, Kt)
        
        #Diagonal is zero
        for i in range(len(w)):
            w[i][i]= 0
        
        return w
   
    # Sample Method 
    def run(self, S):
        file = open("Hopfield.out", "w") 
        i= 0
        prevS= S
        
        file.write(str(prevS))
        file.write("\n")
        
        actS= self.prodW(prevS)
        while (not np.array_equal(prevS, actS) and i < 5000000):
            i= i+1
            prevS= actS;
            
            file.write("Epoch: ")
            file.write(str(i))
            file.write("\n")
            file.write(str(prevS))
            file.write("\n")
            
            actS= self.prodW(prevS)
            
        file.write("Epoch: ") 
        file.write(str(i))
        file.write("\n")
        file.write(str(actS))
        file.write("\n")        
        file.close()
        
        return(actS)
    
    def printW(self):
        print(self.W)
        
    def prodW(self, S):
        return np.sign(np.dot(self.W, S)).astype(int)

In [34]:
toHop= [data[11], data[15], data[19], data[23]]
h= Hopfield(toHop)

Pruebo mandando un elemento de los guardados, y debería devolver el mismo en 0 iteraciones:

In [35]:
print(data[11])
a= h.run(data[11])
print(a)

[1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]
[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
  1]


Pruebo cambiando levemente un elemento guardado, y debería devolver el mismo en un par de iteraciones:

In [36]:
print(data[11])
a= h.run([1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1])
print(a)

[1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]
[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
  1]


Pruebo ingresando la Y, que es parecida a la X:

In [35]:
print(data[23])
a= h.run(data[24])
print(a)

[1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, 1]
4
[ 1. -1. -1. -1.  1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1.
  1. -1.  1. -1. -1. -1.  1.]


In [37]:
L = np.array([1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1])
P= np.array([1, 1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1])
print(L)
print(P)

[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
  1]
[ 1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1
 -1]


In [38]:
matrix= np.array([L])
print(matrix)
print()
matrix= np.concatenate([matrix, [P]])
print(matrix)

[[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
   1]]

[[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
   1]
 [ 1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1
  -1]]


In [39]:
toSend= [data[11], data[15], data[19], data[23]]
print(toSend[0])

[1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1]


In [18]:
toSend= [data[11], data[15], data[19], data[23]]
Kt= np.array([toSend.pop(0)])
while len(toSend) > 0:
    newRow= np.array(toSend.pop(0))
    Kt= np.concatenate([Kt, [newRow]])
K= Kt.T

print(toSend)
print(Kt)
print(K)

[]
[[ 1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1  1  1  1
   1]
 [ 1  1  1  1  1  1 -1 -1 -1  1  1  1  1  1  1  1 -1 -1 -1 -1  1 -1 -1 -1
  -1]
 [ 1  1  1  1  1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1 -1 -1 -1  1 -1
  -1]
 [ 1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1 -1 -1  1 -1  1 -1  1 -1 -1 -1
   1]]
[[ 1  1  1  1]
 [-1  1  1 -1]
 [-1  1  1 -1]
 [-1  1  1 -1]
 [-1  1  1  1]
 [ 1  1 -1 -1]
 [-1 -1 -1  1]
 [-1 -1  1 -1]
 [-1 -1 -1  1]
 [-1  1 -1 -1]
 [ 1  1 -1 -1]
 [-1  1 -1 -1]
 [-1  1  1  1]
 [-1  1 -1 -1]
 [-1  1 -1 -1]
 [ 1  1 -1 -1]
 [-1 -1 -1  1]
 [-1 -1  1 -1]
 [-1 -1 -1  1]
 [-1 -1 -1 -1]
 [ 1  1 -1  1]
 [ 1 -1 -1 -1]
 [ 1 -1  1 -1]
 [ 1 -1 -1 -1]
 [ 1 -1 -1  1]]


In [19]:
toSend= [data[11], data[15], data[19], data[23]]
print(len(toSend.pop(0)))

25


In [47]:
def noisyData(data, prob):
    toRet= []
    for i in range(len(data)):
        rand= np.random.rand()
        if (rand < prob):
            toRet.append(data[i] * -1)
        else:
            toRet.append(data[i])
    return toRet

toTest= noisyData(data[20], 0.4)

In [48]:
print(data[20])
print(toTest)

[1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1]
[1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1]


In [49]:
h.run(toTest)

array([ 1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1,  1,  1,  1,  1,  1])